In [2]:
import matplotlib

%matplotlib inline
# matplotlib.use("Agg")

import mdtraj as md
import numpy as np
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy
from scipy.spatial.distance import squareform
import pandas as pd

# import nglview as nv
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import rdMolDraw2D
from IPython.display import SVG

In [3]:
compound_index = int(snakemake.wildcards.compound_dir)

In [4]:
# load omega conformers
t_omega = md.load(snakemake.input.pdb)
t_omega

In [5]:
t_ref = md.load(snakemake.input.ref_top)
t_ref

In [8]:
ref_mol = Chem.MolFromPDBFile(
    snakemake.input.ref_pdb_equil, removeHs=False, sanitize=False
)
# ref_mol = Chem.MolFromPDBFile(snakemake.input.ref_top, removeHs=False, sanitize=False)
# ref_mol = Chem.MolFromPDBFile("data/interim/refactor-test/24/H2O/1_make_topology/equil.pdb", removeHs=False, sanitize=False)

In [9]:
ref_mol.RemoveAllConformers()

In [10]:
# smi_mol = Chem.MolFromSmiles('CC(C)C[C@H]1C(=O)N[C@H](C(=O)N[C@H](C(=O)N[C@H](C(=O)N[C@H](C(=O)N2CCC[C@H]2C(=O)N1)CO)CC(=O)O)C(C)C)CC(=O)N')
# ref_mol = AllChem.AssignBondOrdersFromTemplate(smi_mol, ref_mol)

In [11]:
ref_mol

In [12]:
ref_mol.GetNumAtoms()

In [13]:
conf_gens = Chem.MolFromPDBFile(
    snakemake.input.pdb, removeHs=False, sanitize=False
)
# conf_gens = AllChem.AssignBondOrdersFromTemplate(smi_mol, conf_gens)

In [14]:
conf_gens

In [15]:
conf_gens.RemoveAllConformers()
conf_gens

In [16]:
conf_gens.GetNumAtoms()

In [17]:
Chem.MolToSmiles(conf_gens)

In [18]:
Chem.MolToSmiles(ref_mol)

In [19]:
AllChem.AssignBondOrdersFromTemplate(ref_mol, conf_gens)

In [20]:
# hit_ats = list(conf_gens.GetSubstructMatch(Chem.RemoveHs(ref_mol)))

In [21]:
hit_ats = list(conf_gens.GetSubstructMatch(ref_mol))

In [22]:
d = rdMolDraw2D.MolDraw2DSVG(500, 500)  # or MolDraw2DCairo to get PNGs
rdMolDraw2D.PrepareAndDrawMolecule(d, conf_gens, highlightAtoms=hit_ats)
d.FinishDrawing()
SVG(d.GetDrawingText())

In [23]:
# Find missing atom indices.
missing = set(range(conf_gens.GetNumAtoms())) - set(hit_ats)

In [24]:
missing

In [25]:
# Now remove these atoms
mw = Chem.RWMol(conf_gens)
for i in missing:
    mw.RemoveAtom(i)

In [26]:
order = list(mw.GetSubstructMatch(ref_mol))
Chem.RenumberAtoms(mw, order)

In [27]:
# Chem.MolToPDBFile(mw, snakemake.output.mol_aligned, flavor=2)

In [28]:
# remove missing atoms in mdtraj object:

In [29]:
not_missing = list(set(range(conf_gens.GetNumAtoms())) - missing)
t_omega_corrected = t_omega.atom_slice(not_missing)

In [30]:
t_omega_corrected.save_pdb(snakemake.output.mol_aligned)